In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-04-14 12:37:29,268 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,FBC Colorado
1,Texas Club 2025
2,Real SCI
3,Gig@ntte
4,PraiaBravaFC


### Buscar IDs dos times no Cartola

In [3]:
nomes_times = ['FBC Colorado', 'FBC Colorado II','FBC Colorado III']#, "GaúchoDaFronteira F.C",'Texas Club 2025', 'Real SCI', 'Gig@ntte', 'PraiaBravaFC', 'OlhaEleAiF.C!', 'Gremiomaniasm', 'Sport Clube PAIM', 'PUXE FC', 'RS Expressões da Arte',
            #    'ZIVI FC', 'O clube do povo Itaqui/Rss', 'F.C. Rei Das Copas', 'Rolo Compressor ZN', 'Taura da Fronteira FCIII', 'Eleis-Itaqui', 'KING LEONN', 'Laranjja Mecannica', 
            #    'Fedato Futebol Clube', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'RHANKA DENTY FC25', 'TEAM LOPES 99', 'pura bucha /botafogo', 'cartola scheuer', 'Remo Santo Ângelo', 
            #    'Analove10 ITAQUI GRANDE!!', 'DM Studio', 'lsauer fc', 'VASCO MARTINS FC', 'KP JUV.', 'BORGES ITAQUI F.C.', 'Profit Soccer', 'Tabajara de Inhaua FC2', 'TIGRE LEON', 
            #    'S.E.R. GRILLO', 'seralex', 'E.C. Bororé', 'MAFRA MARTINS FC', 'BordonFC', 'Tatols Beants F.C', 'FIGUEIRA DA ILHA', 'MauHumor F.C.', 'A Lenda Super Vascão f.c', 
            #    'TATITTA FC', 'HS SPORTS F.C', 'Dom Camillo68', 'mercearia Estrela', 'CosmoCity ZO', 'clarinvalau fc', 'Grêmio imortal 37', 'SERGRILLO', 'Super Vasco f.c', 
            #    'A Lenda Super Vasco F.c', 'Paulo Virgili FC', 'CALOMBO ITAQUI RS', 'Luis lemes inter', 'emer jr fc', 'Cril Futebol Club', 'ITAQUI F.C.', 'TORRESMO COM PINGA', 
            #    'Lá do Itaqui', 'FC Los Castilho', 'KillerColorado', "Atlético Colorado 2021", "Time do S.A.P.O", "SISO FC25", "Grêmio imortal 37", "BORGES ITAQUI F.C."]

# nomes_times = ["EdshooowFC", "Defenders F. C.", "Riacho s6", "VIVOPELAGRACA", "Barbosinha FC", "FairPlay Tiro Curto 3",
#                "28 di Marco EC", "QSITO", "jhullyacesar@", "PRECHEKOS FC", "FBC Colorado"]

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# ➕ Correções manuais
# ids_times["BORGES ITAQUI F.C."] = 3914981
# ids_times["Time do S.A.P.O"] = 12345678  # Substitua pelo ID real

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 3
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [4]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_D.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_D.xlsx


,Nome do Time,ID do Time,Link do Time
0,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
1,FBC Colorado II,1863710,https://cartola.globo.com/#!/time/1863710
2,FBC Colorado III,5749052,https://cartola.globo.com/#!/time/5749052


### Gerar o dicionário ID -> Nome do Time

In [5]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{186283: 'FBC Colorado',
 1863710: 'FBC Colorado II',
 5749052: 'FBC Colorado III'}

In [6]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [7]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

,Rodada 1,Rodada 2,Rodada 3
FBC Colorado,64.00,72.69,81.52
FBC Colorado II,NaN,NaN,80.75
FBC Colorado III,NaN,NaN,88.42
Lider_Rodada,FBC Colorado,FBC Colorado,FBC Colorado III


### Função para definir a classificação dos times

In [8]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [9]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_D.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série D"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head())

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,FBC Colorado,Tatols Beants F.C,6,15,Série D
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17,Série D
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10,Série D
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16,Série D
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1,Série D


ValueError: No objects to concatenate

In [ ]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [ ]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_a.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_a.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_D.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_D.js'

In [ ]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_D.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série D"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_D.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_D.js gerado com sucesso!")


✅ Arquivo confrontos_serie_D.js gerado com sucesso!


In [ ]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_D.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,FBC Colorado,Tatols Beants F.C,6,15
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1


In [ ]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série D"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série A por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [ ]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_D.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_D.js gerado com sucesso.")


✅ Arquivo resultados_serie_D.js gerado com sucesso.


# Análises de Pontuações de Jogadores

## Obter Pontuações por Rodada

In [10]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

# Função para verificar se o campeonato já começou (há pontuação na rodada 1)
def campeonato_comecou(ids_times):
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True  # Campeonato já começou
        except cartolafc.errors.CartolaFCError:
            continue
    return False  # Nenhuma pontuação encontrada

# Verifica se o campeonato começou
if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")

    total_rodadas = 38  # Número total de rodadas do campeonato
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = 0  # ou None se preferir

else:
    # Função para obter a pontuação de um time em todas as rodadas
    def obter_pontuacao_por_rodada(time_id, rodada_atual):
        pontuacoes = {}
        for rodada in range(1, rodada_atual):
            try:
                time_rodada = api.time(time_id=time_id, rodada=rodada)
                pontuacoes[rodada] = time_rodada.ultima_pontuacao
            except cartolafc.errors.CartolaFCError as e:
                print(f"Erro ao acessar a pontuação da rodada {rodada} para o time {time_id}: {e}")
                pontuacoes[rodada] = None
        return pontuacoes

    # DataFrame para armazenar as pontuações por rodada
    df_pontuacoes = pd.DataFrame()

    # Obter as pontuações por rodada para cada time
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(time_id, rodada_atual)
        df_pontuacoes[nome] = pd.Series(pontuacoes)

    # Transpor o DataFrame para ter rodadas como colunas
    df_pontuacoes = df_pontuacoes.transpose()

    # Renomear as colunas para representar as rodadas
    df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]

    # Adicionar a linha "Lider_Rodada"
    lider_rodada = df_pontuacoes.idxmax()
    # df_pontuacoes.loc['Lider_Rodada'] = lider_rodada

df_pontuacoes = df_pontuacoes.sort_values(by="Rodada 1", ascending=False)

# Exibir o dataframe
display(df_pontuacoes)

# Salvar o dataframe em um arquivo Excel
df_pontuacoes.to_excel('DADOS_API_pontuacoes_por_rodada.xlsx')


,Rodada 1,Rodada 2,Rodada 3
FBC Colorado,64.00,72.69,81.52
FBC Colorado II,NaN,NaN,80.75
FBC Colorado III,NaN,NaN,88.42


## Obter Escalação de um  Time em uma determinada Rodada

In [11]:
# Função para obter a escalação de um time em uma rodada específica, incluindo o clube do jogador
def obter_escalacao_time(time_id, rodada):
    try:
        time_rodada = api.time(time_id=time_id, rodada=rodada)
        if not time_rodada.atletas:
            print(f"⚠️ Nenhum jogador encontrado para o time {time_id} na rodada {rodada}. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.")
            return []
        jogadores = [
            {
                'Nome': atleta.apelido,
                'Posição': atleta.posicao.nome,
                'Pontos': atleta.pontos,
                'Clube': atleta.clube.nome
            }
            for atleta in time_rodada.atletas
        ]
        return jogadores
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao acessar a escalação da rodada {rodada} para o time {time_id}: {e}")
        return []

# Parâmetros de entrada
rodada_especifica = 2  # Substitua pela rodada desejada
nome_time = 'FBC Colorado'  # Substitua pelo nome do time desejado

# Verificação se o time está no dicionário
if nome_time not in ids_times:
    print(f"❌ O time '{nome_time}' não foi encontrado no dicionário de IDs.")
else:
    time_cartola = ids_times[nome_time]
    time_escalacao = obter_escalacao_time(time_cartola, rodada_especifica)

    if time_escalacao:
        df_escalacao = pd.DataFrame(time_escalacao)
        # df_escalacao.sort_values(by='Posição', inplace=True)  # Ou por 'Pontos' se preferir

        df_escalacao_pontuacao = df_escalacao['Pontos'].sum()

        print(f'✅ Time do Cartola: {nome_time}')
        print(f'📊 Pontuação na Rodada {rodada_especifica}: {df_escalacao_pontuacao:.2f}')
        display(df_escalacao)
    else:
        print("⚠️ Escalação vazia ou não disponível.")



✅ Time do Cartola: FBC Colorado
📊 Pontuação na Rodada 2: 66.29


,Nome,Posição,Pontos,Clube
0,Anthoni,Goleiro,6.30,INT
1,Renê,Lateral,2.40,FLU
2,Escobar,Lateral,6.40,SAN
3,Lyanco,Zagueiro,3.10,CAM
4,Léo Ortiz,Zagueiro,0.00,FLA
5,Arias,Meia,2.00,FLU
6,Gabriel Bontempo,Meia,6.00,SAN
7,Carrillo,Meia,3.60,COR
8,Hulk,Atacante,3.60,CAM
9,Igor Jesus,Atacante,10.60,BOT


## Obter Escalações Gerais

In [12]:
import time  # opcional, para colocar pausa entre requisições


# Função para obter escalações de todos os times em todas as rodadas
def obter_escalacoes_gerais(ids_times, rodada_atual):
    escalacoes_gerais = []

    for nome_time, time_id in ids_times.items():
        print(f"🔄 Coletando escalações para o time: {nome_time}")
        for rodada in range(1, rodada_atual + 1):
            escalação = obter_escalacao_time(time_id, rodada)

            if not escalação:
                print(f"⚠️ Rodada {rodada}: sem escalação disponível para {nome_time}.")
                continue

            for jogador in escalação:
                jogador['Time Cartola'] = nome_time
                jogador['Rodada'] = rodada
                escalacoes_gerais.append(jogador)

            # Opcional: pausa para evitar sobrecarga de requisições (API friendly)
            time.sleep(0.2)

    return escalacoes_gerais

# Obter escalações gerais
escalacoes_gerais = obter_escalacoes_gerais(ids_times, rodada_atual)

# Armazenar escalações em um DataFrame
df_escalacoes = pd.DataFrame(escalacoes_gerais)

if df_escalacoes.empty:
    print("⚠️ Nenhuma escalação disponível até o momento. O DataFrame está vazio.")
else:
    # Organizar colunas (opcional)
    colunas_ordenadas = ['Rodada', 'Time Cartola', 'Nome', 'Posição', 'Clube', 'Pontos']
    df_escalacoes = df_escalacoes[colunas_ordenadas]

    # Salvar o DataFrame em um arquivo Excel
    df_escalacoes.to_excel('DADOS_API_escalacoes_gerais.xlsx', index=False)

    # Visualizar primeiras linhas
    display(df_escalacoes.head())

🔄 Coletando escalações para o time: FBC Colorado
🔄 Coletando escalações para o time: FBC Colorado II
⚠️ Nenhum jogador encontrado para o time 1863710 na rodada 1. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.
⚠️ Rodada 1: sem escalação disponível para FBC Colorado II.
⚠️ Nenhum jogador encontrado para o time 1863710 na rodada 2. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.
⚠️ Rodada 2: sem escalação disponível para FBC Colorado II.
🔄 Coletando escalações para o time: FBC Colorado III
⚠️ Nenhum jogador encontrado para o time 5749052 na rodada 1. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.
⚠️ Rodada 1: sem escalação disponível para FBC Colorado III.
⚠️ Nenhum jogador encontrado para o time 5749052 na rodada 2. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.
⚠️ Rodada 2: sem escalação disponível para FBC Colorado III.


,Rodada,Time Cartola,Nome,Posição,Clube,Pontos
0,1,FBC Colorado,Weverton,Goleiro,PAL,11.50
1,1,FBC Colorado,Renê,Lateral,FLU,5.00
2,1,FBC Colorado,Juninho Capixaba,Lateral,RBB,7.40
3,1,FBC Colorado,Sabino,Zagueiro,SAO,7.30
4,1,FBC Colorado,Léo Ortiz,Zagueiro,FLA,0.60


## Função para listar os jogadores mais escalados por posição

In [13]:
# Função para listar os jogadores mais escalados por posição
def listar_jogadores_mais_escalados(df, n_goleiros=3, n_zagueiros=4, n_laterais=4, n_meias=5, n_atacantes=5):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhuma escalação disponível ainda.")
        return pd.DataFrame()

    # Agrupar por nome, posição e clube, somando os pontos e contando a quantidade de vezes escalados
    df_agrupado = df.groupby(['Nome', 'Posição', 'Clube'], as_index=False).agg(
        Pontos=('Pontos', 'sum'),
        Quantidade=('Nome', 'size')
    )

    # Filtrar e ordenar para cada posição
    goleiros = df_agrupado[df_agrupado['Posição'] == 'Goleiro'].sort_values(by='Quantidade', ascending=False).head(n_goleiros)
    zagueiros = df_agrupado[df_agrupado['Posição'] == 'Zagueiro'].sort_values(by='Quantidade', ascending=False).head(n_zagueiros)
    laterais = df_agrupado[df_agrupado['Posição'] == 'Lateral'].sort_values(by='Quantidade', ascending=False).head(n_laterais)
    meias = df_agrupado[df_agrupado['Posição'] == 'Meia'].sort_values(by='Quantidade', ascending=False).head(n_meias)
    atacantes = df_agrupado[df_agrupado['Posição'] == 'Atacante'].sort_values(by='Quantidade', ascending=False).head(n_atacantes)

    # Concatenar os resultados em um único dataframe
    jogadores_mais_escalados = pd.concat([goleiros, zagueiros, laterais, meias, atacantes])

    # colunas = ['Nome', 'Posição', 'Clube', 'Quantidade', 'Pontos']
    colunas = ['Nome', 'Posição', 'Clube', 'Quantidade']
    jogadores_mais_escalados = jogadores_mais_escalados[colunas]

    return jogadores_mais_escalados

# Execução protegida com verificação
jogadores_mais_escalados = listar_jogadores_mais_escalados(df_escalacoes)

if not jogadores_mais_escalados.empty:
    print("📋 Jogadores mais escalados por posição:")
    display(jogadores_mais_escalados)

📋 Jogadores mais escalados por posição:


,Nome,Posição,Clube,Quantidade
1,Anthoni,Goleiro,INT,3
39,Weverton,Goleiro,PAL,3
33,Ronaldo,Goleiro,BAH,2
27,Léo Ortiz,Zagueiro,FLA,6
0,Abner,Zagueiro,JUV,2
20,Junior Alonso,Zagueiro,CAM,2
22,Kanu,Zagueiro,BAH,2
19,Juninho Capixaba,Lateral,RBB,7
28,Natanael,Lateral,CAM,4
23,Luciano Juba,Lateral,BAH,2


### Buscando pontuação em tempo real na API

In [17]:
import requests
import pandas as pd

# Consulta o status do mercado
status_url = "https://api.cartola.globo.com/mercado/status"
status_response = requests.get(status_url)
status_data = status_response.json()

mercado_status = status_data.get('status_mercado', None)
print("Status do Mercado:", mercado_status)

if mercado_status == 1:
    print("O mercado está aberto para escalações. A rodada ainda não começou, então as pontuações parciais não estarão disponíveis.")

elif mercado_status == 2:
    print("A rodada está em andamento. Realizando consulta de pontuações parciais...")

    url = "https://api.cartola.globo.com/atletas/pontuados"
    response = requests.get(url)

    if response.status_code == 200 and response.text.strip():
        try:
            data = response.json()
            jogadores = data.get('atletas', {})
            clubes = data.get('clubes', {})
            posicoes = data.get('posicoes', {})

            lista_jogadores = []
            for atleta_id, info in jogadores.items():
                clube_nome = clubes[str(info['clube_id'])]['nome']
                posicao_nome = posicoes[str(info['posicao_id'])]['nome']
                lista_jogadores.append({
                    'ID': atleta_id,
                    'Nome': info['apelido'],
                    'Clube': clube_nome,
                    'Posição': posicao_nome,
                    'Pontuação': info['pontuacao']
                })

            df_parciais = pd.DataFrame(lista_jogadores)
            print(df_parciais.head())

        except Exception as e:
            print("Erro ao interpretar JSON:", e)
    else:
        print("⚠️ Nenhum dado de pontuação parcial disponível.")

elif mercado_status == 3:
    print("O mercado está atualizando os dados pós-rodada.")
else:
    print("Status do mercado desconhecido. Verifique se a API está operando corretamente.")


Status do Mercado: 1
O mercado está aberto para escalações. A rodada ainda não começou, então as pontuações parciais não estarão disponíveis.


In [18]:
import requests
import pandas as pd

def buscar_id_por_nome(nome_time):
    url_busca = f"https://api.cartola.globo.com/times?q={nome_time}"
    resp = requests.get(url_busca)
    if resp.status_code == 200:
        times = resp.json()
        if times:
            time_encontrado = times[0]  # pega o primeiro resultado
            print(f"🔍 Time encontrado: {time_encontrado['nome']} (ID: {time_encontrado['time_id']})")
            return time_encontrado['time_id']
        else:
            print("❌ Nenhum time encontrado com esse nome.")
            return None
    else:
        print("❌ Erro ao buscar time pelo nome.")
        return None

def obter_pontuacao_parcial_time(identificador):
    # Verifica se o identificador é numérico (ID) ou nome (string)
    if isinstance(identificador, int) or identificador.isdigit():
        time_id = int(identificador)
    else:
        time_id = buscar_id_por_nome(identificador)
        if time_id is None:
            return

    # Passo 1: Consulta os jogadores escalados
    url_escalacao = f"https://api.cartola.globo.com/time/id/{time_id}"
    resp_time = requests.get(url_escalacao)
    
    if resp_time.status_code != 200:
        print("❌ Erro ao buscar dados do time.")
        return
    
    dados_time = resp_time.json()
    atletas_escalação = dados_time.get("atletas", [])

    if not atletas_escalação:
        print("❌ Nenhum jogador escalado ou mercado já fechado.")
        return

    # Passo 2: Consulta as pontuações parciais
    url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
    resp_parciais = requests.get(url_parciais)

    if resp_parciais.status_code != 200:
        print("❌ Erro ao buscar pontuações parciais.")
        return

    data_parciais = resp_parciais.json()
    pontuados = data_parciais.get("atletas", {})
    clubes = data_parciais.get("clubes", {})
    posicoes = data_parciais.get("posicoes", {})

    # Passo 3: Monta DataFrame com as pontuações dos jogadores escalados
    lista = []
    for atleta in atletas_escalação:
        atleta_id = atleta['atleta_id']
        if str(atleta_id) in pontuados:
            dados = pontuados[str(atleta_id)]
            nome = dados['apelido']
            pontuacao = dados['pontuacao']
            clube = clubes[str(dados['clube_id'])]['nome']
            posicao = posicoes[str(dados['posicao_id'])]['nome']
            lista.append({
                'Nome': nome,
                'Clube': clube,
                'Posição': posicao,
                'Pontuação': pontuacao
            })

    df = pd.DataFrame(lista)
    print(df)
    print(f"\n➡️ Pontuação Parcial do Time: {df['Pontuação'].sum():.2f} pontos")

# 🟢 Exemplo de uso:
# Com nome
# obter_pontuacao_parcial_time("FBC Colorado")

# Com ID
obter_pontuacao_parcial_time("3708025")


❌ Erro ao buscar pontuações parciais.


In [19]:
import pandas as pd
import requests
from time import sleep

# === 1. Lê o arquivo com os times da liga ===
arquivo = "links_times_cartola_liga_serie_D.xlsx"
df_times = pd.read_excel(arquivo)

# Verifica se as colunas esperadas existem
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. Consulta pontuações parciais ===
print("🔄 Consultando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 3. Loop para todos os times ===
pontuacoes_liga = []

for _, row in df_times.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])
    
    print(f"🔍 Buscando escalação de: {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)
    
    if resp_time.status_code != 200:
        print(f"❌ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    pontuacao_total = 0
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            pontuacao_total += pontuados[str(atleta_id)]['pontuacao']
    
    pontuacoes_liga.append({
        'Nome do Time': nome_time,
        'ID do Time': id_time,
        'Pontuação Parcial': round(pontuacao_total, 2)
    })
    
    # Boa prática: esperar um pouco para não sobrecarregar a API
    sleep(0.5)

# === 4. Monta DataFrame final com pontuação da liga ===
df_resultado = pd.DataFrame(pontuacoes_liga)
df_resultado = df_resultado.sort_values(by='Pontuação Parcial', ascending=False).reset_index(drop=True)

🔄 Consultando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

In [20]:
# Exibe o resultado
display(df_resultado.head(30))

NameError: name 'df_resultado' is not defined

In [21]:
import pandas as pd
import requests
import time
from time import sleep

# === 1. Lê o arquivo Excel ===
arquivo = "links_times_cartola_liga_serie_D.xlsx"
df_times = pd.read_excel(arquivo)

# Verifica colunas
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. FILTRO: adicione os nomes desejados aqui ===
# Deixe a lista vazia para consultar todos

times_desejados = [
    "FBC Colorado",    
    "Gig@ntte",
    "E.C. Bororé",
    "Real SCI",    
    "GaúchoDaFronteira F.C",
    "OlhaEleAiF.C!",
    "KING LEONN",
    "Texas Club 2025",
    "FBC Colorado II",
    "FBC Colorado III"
]

# times_desejados = ["EdshooowFC", "Defenders F. C.", "Riacho s6", "VIVOPELAGRACA", "Barbosinha FC", "FairPlay Tiro Curto 3",
#                "28 di Marco EC", "QSITO", "jhullyacesar@", "PRECHEKOS FC", "FBC Colorado"]

# Aplica o filtro (ignora maiúsculas/minúsculas)
if times_desejados:
    df_times = df_times[df_times['Nome do Time'].str.lower().isin([t.lower() for t in times_desejados])]
    if df_times.empty:
        raise ValueError("❌ Nenhum dos times filtrados foi encontrado no arquivo.")

# === 3. Consulta pontuações parciais ===
print("🔄 Buscando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 4. Loop pelos times selecionados ===
pontuacoes_liga = []

for _, row in df_times.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    pontuacao_total = 0
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            pontuacao_total += pontuados[str(atleta_id)]['pontuacao']

    pontuacoes_liga.append({
        'Nome do Time': nome_time,
        'ID do Time': id_time,
        'Pontuação Parcial': round(pontuacao_total, 2)
    })

    sleep(0.5)

# === 5. Mostra resultado ordenado ===
df_resultado = pd.DataFrame(pontuacoes_liga)
df_resultado = df_resultado.sort_values(by='Pontuação Parcial', ascending=False).reset_index(drop=True)

# Exibe o resultado
display(df_resultado)

🔄 Buscando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

In [22]:
import pandas as pd
import requests

# === 1. Lê o arquivo Excel ===
arquivo = "links_times_cartola_liga_serie_D.xlsx"
df_times = pd.read_excel(arquivo)

# Verifica colunas
colunas_esperadas = ['Nome do Time', 'ID do Time']
if not all(col in df_times.columns for col in colunas_esperadas):
    raise ValueError(f"O arquivo deve conter as colunas: {colunas_esperadas}")

# === 2. Defina os 3 times que deseja analisar ===
times_desejados = [
    "FBC Colorado",    
    "Gig@ntte",
    "E.C. Bororé",
    "Real SCI",    
    "GaúchoDaFronteira F.C",
    "OlhaEleAiF.C!",
    "KING LEONN",
    "Texas Club 2025",
    "FBC Colorado II",
    "FBC Colorado III"
]

# times_desejados = ["EdshooowFC", "Defenders F. C.", "Riacho s6", "VIVOPELAGRACA", "Barbosinha FC", "FairPlay Tiro Curto 3",
#                "28 di Marco EC", "QSITO", "jhullyacesar@", "PRECHEKOS FC", "FBC Colorado"]


# Aplica o filtro ignorando maiúsculas/minúsculas
df_selecionados = df_times[df_times['Nome do Time'].str.lower().isin([t.lower() for t in times_desejados])]
if df_selecionados.empty:
    raise ValueError("❌ Nenhum dos times desejados foi encontrado.")

# === 3. Consulta pontuações parciais ===
print("🔄 Buscando pontuações parciais...")
url_parciais = "https://api.cartola.globo.com/atletas/pontuados"
resp_parciais = requests.get(url_parciais)
if resp_parciais.status_code != 200:
    raise Exception("❌ Erro ao buscar pontuações parciais.")

data_parciais = resp_parciais.json()
pontuados = data_parciais.get("atletas", {})
clubes = data_parciais.get("clubes", {})
posicoes = data_parciais.get("posicoes", {})

# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    for jogador in atletas:
        atleta_id = jogador['atleta_id']
        if str(atleta_id) in pontuados:
            info = pontuados[str(atleta_id)]
            nome = info['apelido']
            pontuacao = info['pontuacao']
            clube = clubes[str(info['clube_id'])]['nome']
            posicao = posicoes[str(info['posicao_id'])]['nome']
            lista_jogadores.append({
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao,
                'Pontuação': pontuacao
            })

    df_jogadores = pd.DataFrame(lista_jogadores).sort_values(by='Pontuação', ascending=False).reset_index(drop=True)
    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")


🔄 Buscando pontuações parciais...


Exception: ❌ Erro ao buscar pontuações parciais.

### Escalações Completas dos Times Escolhidos na Rodada

In [23]:
# Ordem desejada das posições
ordem_posicoes = {
    'Goleiro': 1,
    'Lateral': 2,
    'Zagueiro': 3,
    'Meia': 4,
    'Atacante': 5,
    'Técnico': 6
}

# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        nome = jogador.get('apelido', 'Desconhecido')
        posicao_id = str(jogador.get('posicao_id', '0'))
        clube_id = str(jogador.get('clube_id', '0'))

        pontuacao = pontuados.get(atleta_id, {}).get("pontuacao", 0.0)
        clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
        posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

        lista_jogadores.append({
            'Nome do Jogador': nome,
            'Clube': clube,
            'Posição': posicao,
            'Pontuação': pontuacao,
            'Ordem Posição': ordem_posicoes.get(posicao, 99)  # valor alto para posições desconhecidas
        })

    # Ordena por ordem da posição e, em seguida, por pontuação (descrescente)
    df_jogadores = pd.DataFrame(lista_jogadores).sort_values(
        by=['Ordem Posição', 'Pontuação'], ascending=[True, False]
    ).reset_index(drop=True)

    df_jogadores.drop(columns=['Ordem Posição'], inplace=True)

    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")



🔍 FBC Colorado (ID: 186283)


NameError: name 'pontuados' is not defined

### Jogadores mais Escalados por Posição dos Times Selecionados

In [24]:
from collections import defaultdict, Counter

def jogadores_mais_escalados_por_posicao(df_times, pontuados, clubes, posicoes):
    """
    Recebe o DataFrame com times, dados de pontuação parcial, clubes e posições.
    Retorna os jogadores mais escalados por posição.
    """
    escalações = []

    for _, row in df_times.iterrows():
        nome_time = row['Nome do Time']
        id_time = int(row['ID do Time'])

        url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
        resp_time = requests.get(url_escalacao)

        if resp_time.status_code != 200:
            print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
            continue

        dados_time = resp_time.json()
        atletas = dados_time.get("atletas", [])

        for jogador in atletas:
            atleta_id = str(jogador['atleta_id'])
            nome = jogador.get('apelido', 'Desconhecido')
            posicao_id = str(jogador.get('posicao_id', '0'))
            clube_id = str(jogador.get('clube_id', '0'))

            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")
            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")

            escalações.append((nome, posicao, clube, atleta_id))

    # Contar jogadores por posição
    contagem = defaultdict(Counter)
    for nome, posicao, clube, atleta_id in escalações:
        chave = f"{nome} ({clube})"
        contagem[posicao][chave] += 1

    # Gerar DataFrames por posição
    dataframes_posicoes = {}
    for posicao, counter in contagem.items():
        df = pd.DataFrame(counter.items(), columns=["Jogador", "Escalações"]).sort_values(by="Escalações", ascending=False)
        dataframes_posicoes[posicao] = df

    return dataframes_posicoes


In [25]:
# Após buscar os dados das pontuações parciais
dataframes_por_posicao = jogadores_mais_escalados_por_posicao(df_selecionados, pontuados, clubes, posicoes)

# Exibir resultados
for posicao in ['Goleiro', 'Lateral', 'Zagueiro', 'Meia', 'Atacante', 'Técnico']:
    print(f"\n📌 {posicao}s mais escalados:")
    if posicao in dataframes_por_posicao:
        print(dataframes_por_posicao[posicao])
    else:
        print("Nenhum jogador escalado nessa posição.")


NameError: name 'pontuados' is not defined

### Jogadores com Scouts

In [26]:
# === 4. Exibe os jogadores escalados e as pontuações parciais por time ===
for _, row in df_selecionados.iterrows():
    nome_time = row['Nome do Time']
    id_time = int(row['ID do Time'])

    print(f"\n🔍 {nome_time} (ID: {id_time})")
    url_escalacao = f"https://api.cartola.globo.com/time/id/{id_time}"
    resp_time = requests.get(url_escalacao)

    if resp_time.status_code != 200:
        print(f"⚠️ Erro ao buscar dados do time {nome_time}. Pulando...")
        continue

    dados_time = resp_time.json()
    atletas = dados_time.get("atletas", [])

    lista_jogadores = []
    scouts_unicos = set()

    # Primeiro, coletamos todos os scouts únicos
    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        scout = pontuados.get(atleta_id, {}).get("scout") or {}
        scouts_unicos.update(scout.keys())

    for jogador in atletas:
        atleta_id = str(jogador['atleta_id'])
        nome = jogador.get('apelido', 'Desconhecido')
        posicao_id = str(jogador.get('posicao_id', '0'))
        clube_id = str(jogador.get('clube_id', '0'))

        pontuacao = pontuados.get(atleta_id, {}).get("pontuacao", 0.0)
        clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
        posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")
        scout = pontuados.get(atleta_id, {}).get("scout") or {}  # ← Aqui está a correção

        jogador_data = {
            'Nome do Jogador': nome,
            'Clube': clube,
            'Posição': posicao,
            'Pontuação': pontuacao
        }

        for s in scouts_unicos:
            jogador_data[s] = scout.get(s, 0)

        lista_jogadores.append(jogador_data)


    df_jogadores = pd.DataFrame(lista_jogadores)

    # Ordena por posição e pontuação
    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df_jogadores['Ordem Posição'] = df_jogadores['Posição'].map(ordem_posicoes)
    df_jogadores = df_jogadores.sort_values(by=['Ordem Posição', 'Pontuação'], ascending=[True, False]).reset_index(drop=True)
    df_jogadores.drop(columns=['Ordem Posição'], inplace=True)

    total = df_jogadores['Pontuação'].sum()

    print(df_jogadores)
    print(f"➡️ Total do time {nome_time}: {total:.2f} pontos")



🔍 FBC Colorado (ID: 186283)


NameError: name 'pontuados' is not defined

### Análise do Scout de todos os jogadores do Cartola

In [27]:
from collections import defaultdict
import pandas as pd
import requests
import time

def coletar_scouts_totais_geral(rodada_final=38):
    scouts_agrupados = defaultdict(lambda: defaultdict(int))
    info_jogadores = {}
    todos_scouts = set()

    for rodada in range(1, rodada_final + 1):
        print(f"🔄 Coletando scouts da rodada {rodada}")
        url = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
        resp = requests.get(url)

        if resp.status_code != 200:
            print(f"⚠️ Erro na rodada {rodada}, pulando...")
            continue

        data = resp.json()
        atletas = data.get("atletas", {})
        clubes = data.get("clubes", {})
        posicoes = data.get("posicoes", {})

        for atleta_id, dados in atletas.items():
            nome = dados.get('apelido', 'Desconhecido')
            clube_id = str(dados.get('clube_id', '0'))
            posicao_id = str(dados.get('posicao_id', '0'))
            scout = dados.get("scout") or {}

            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

            chave_jogador = f"{atleta_id}"

            # Salva dados básicos
            info_jogadores[chave_jogador] = {
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao
            }

            for s, valor in scout.items():
                scouts_agrupados[chave_jogador][s] += valor
                todos_scouts.add(s)

        time.sleep(0.4)  # evitar sobrecarga na API

    # Monta o DataFrame final
    jogadores_final = []
    for atleta_id, dados_base in info_jogadores.items():
        linha = dados_base.copy()
        for s in todos_scouts:
            linha[s] = scouts_agrupados[atleta_id].get(s, 0)
        jogadores_final.append(linha)

    df = pd.DataFrame(jogadores_final)

    # Ordena por posição e nome
    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df['Ordem Posição'] = df['Posição'].map(ordem_posicoes)
    df = df.sort_values(by=['Ordem Posição', 'Nome do Jogador']).drop(columns=['Ordem Posição'])

    return df


In [28]:
df_scouts_geral = coletar_scouts_totais_geral(rodada_final=3)


🔄 Coletando scouts da rodada 1
🔄 Coletando scouts da rodada 2
🔄 Coletando scouts da rodada 3


In [29]:
for pos in df_scouts_geral['Posição'].unique():
    print(f"\n📌 {pos}s:")
    display(df_scouts_geral[df_scouts_geral['Posição'] == pos])



📌 Goleiros:


,Nome do Jogador,Clube,Posição,PC,CV,GC,GS,I,DE,CA,PS,DP,G,FF,FT,V,PP,SG,A,FD,DS,FS,FC
209,Alex Muralha,MIR,Goleiro,0,0,0,4,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,Anthoni,INT,Goleiro,0,0,0,1,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0
404,Brenno,FOR,Goleiro,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
210,Bruno Ferreira,CEA,Goleiro,0,0,0,4,0,10,1,0,0,0,0,0,0,0,1,0,0,0,1,1
278,Caíque França,SPT,Goleiro,0,0,0,5,0,8,0,0,0,0,0,0,0,0,1,0,0,0,0,0
294,Cleiton,RBB,Goleiro,0,0,0,4,0,9,1,0,0,0,0,0,0,0,1,0,0,0,0,0
170,Cássio,CRU,Goleiro,0,0,0,5,0,11,2,0,1,0,0,0,0,0,0,0,0,0,0,0
199,Everson,CAM,Goleiro,0,0,0,4,0,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0
166,Fábio,FLU,Goleiro,0,0,0,3,0,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0
188,Gabriel,VIT,Goleiro,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0



📌 Laterals:


,Nome do Jogador,Clube,Posição,PC,CV,GC,GS,I,DE,CA,PS,DP,G,FF,FT,V,PP,SG,A,FD,DS,FS,FC
89,Aguirre,INT,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,0,4
177,Alan Ruschel,JUV,Lateral,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,4,1,1
176,Alex Sandro,FLA,Lateral,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,7,4,9
201,Alex Telles,BOT,Lateral,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,1,2,3,8
91,Andrés Hurtado,RBB,Lateral,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,Vanderlan,PAL,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,Vitinho,BOT,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,5,0,1
219,Wendell,SAO,Lateral,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,6,3
110,Wesley,FLA,Lateral,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,5,4,6



📌 Zagueiros:


...

,Nome do Jogador,Clube,Posição,PC,CV,GC,GS,I,DE,CA,PS,DP,G,FF,FT,V,PP,SG,A,FD,DS,FS,FC
45,Acevedo,BAH,Meia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,Adson,VAS,Meia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
169,Alan Franco,CAM,Meia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2
179,Alan Patrick,INT,Meia,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,0,1
242,Alex Santana,COR,Meia,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Walace,CRU,Meia,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2
230,Wellington Rato,VIT,Meia,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1
226,Willian Oliveira,VIT,Meia,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,5,2
215,Yago Pikachu,FOR,Meia,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,1,5,2,1



📌 Atacantes:


,Nome do Jogador,Clube,Posição,PC,CV,GC,GS,I,DE,CA,PS,DP,G,FF,FT,V,PP,SG,A,FD,DS,FS,FC
8,Ademir,BAH,Atacante,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,6,3
371,Alejandro Martínez,CEA,Atacante,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,1,2
375,Alex Teixeira,VAS,Atacante,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,0,2
348,Allanzinho,FOR,Atacante,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,2
421,Alysson,GRE,Atacante,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Wesley,INT,Atacante,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,2,6,7,2
378,Willian José,BAH,Atacante,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,2,0
0,Yuri Alberto,COR,Atacante,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,1,2,0,3,3
259,Ángel Romero,COR,Atacante,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2



📌 Técnicos:


,Nome do Jogador,Clube,Posição,PC,CV,GC,GS,I,DE,CA,PS,DP,G,FF,FT,V,PP,SG,A,FD,DS,FS,FC
88,Abel Ferreira,PAL,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
171,Cuca,CAM,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
143,Fernando Seabra,RBB,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
152,Filipe Luís,FLA,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
191,Fábio Carille,VAS,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
74,Fábio Matias,JUV,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
202,Gustavo Quinteros,GRE,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
43,Juan Vojvoda,FOR,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
262,Leonardo Jardim,CRU,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
335,Luis Zubeldía,SAO,Técnico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
import pandas as pd
from collections import defaultdict
import requests
import time
import json
import os

# Função para coletar os scouts totais de todos os jogadores do Cartola
def coletar_scouts_totais_geral(rodada_final=3):
    scouts_agrupados = defaultdict(lambda: defaultdict(int))
    info_jogadores = {}
    todos_scouts = set()

    for rodada in range(1, rodada_final + 1):
        url = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
        resp = requests.get(url)
        if resp.status_code != 200:
            continue

        data = resp.json()
        atletas = data.get("atletas", {})
        clubes = data.get("clubes", {})
        posicoes = data.get("posicoes", {})

        for atleta_id, dados in atletas.items():
            nome = dados.get('apelido', 'Desconhecido')
            clube_id = str(dados.get('clube_id', '0'))
            posicao_id = str(dados.get('posicao_id', '0'))
            scout = dados.get("scout") or {}

            clube = clubes.get(clube_id, {}).get("nome", "Desconhecido")
            posicao = posicoes.get(posicao_id, {}).get("nome", "Desconhecido")

            chave_jogador = f"{atleta_id}"

            info_jogadores[chave_jogador] = {
                'Nome do Jogador': nome,
                'Clube': clube,
                'Posição': posicao
            }

            for s, valor in scout.items():
                scouts_agrupados[chave_jogador][s] += valor
                todos_scouts.add(s)

        time.sleep(0.4)

    jogadores_final = []
    for atleta_id, dados_base in info_jogadores.items():
        linha = dados_base.copy()
        for s in todos_scouts:
            linha[s] = scouts_agrupados[atleta_id].get(s, 0)
        jogadores_final.append(linha)

    df = pd.DataFrame(jogadores_final)

    ordem_posicoes = {
        'Goleiro': 1,
        'Lateral': 2,
        'Zagueiro': 3,
        'Meia': 4,
        'Atacante': 5,
        'Técnico': 6
    }
    df['Ordem Posição'] = df['Posição'].map(ordem_posicoes)
    df = df.sort_values(by=['Ordem Posição', 'Nome do Jogador']).drop(columns=['Ordem Posição'])

    return df

# Coleta os dados
df_scouts_geral = coletar_scouts_totais_geral(rodada_final=3)

# Exporta para Excel com abas por posição
excel_path = "scouts_totais_por_posicao.xlsx"
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    for posicao, grupo in df_scouts_geral.groupby("Posição"):
        grupo.to_excel(writer, sheet_name=posicao[:31], index=False)

# Exporta para arquivo .js
# Agrupa por posição e transforma em dicionário
js_dict = {
    posicao: grupo.dropna(axis=1, how='all').to_dict(orient="records")
    for posicao, grupo in df_scouts_geral.groupby("Posição")
}

# Salva como um arquivo .js válido
js_path = "scouts_totais.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write("const scoutsTotais = ")
    json.dump(js_dict, f, ensure_ascii=False, indent=2)
    f.write(";")


excel_path, js_path


('scouts_totais_por_posicao.xlsx', 'scouts_totais.js')